Here I'm making a better non-parametric tree of only a subset of sequences from the bigger v2g5 tree.
The aim is to get a better resolution on the disctiotion between JOX, ANS and FLS clades.
This distinction holds up in the bigger parametric tree methods reproducibly, although not all nodes of interest have sufficient bootstrap support.
With a non-parametric method, I aim to get a better idea of how reliable these nodes are.

**data**

I'm working with a subset of sequences of the JOX, ANS and FLS clades, retrieved from the v1g3 and the v2g5 trees. These subsets are redundant so they should be deduplicated first.

**approach**
I'll get the sequences from previous allignments, build a mafft tree, trim, attampt prank indelrealignment with a iqtree guide-tree, and then build a non-parametric tree with 200 bootstrap runs.

deduplicate the subset list:

In [10]:
sort data/v2g5_subset-ids.txt | uniq | tr ' ' _ > data/temp.txt
mv data/temp.txt data/v2g5_subset-ids.txt

: 1

Make a fasta from this subset list

In [14]:
grep -f data/v2g5_subset-ids.txt \
     -A 1 \
     --no-group-sep \
     data/orthogroup_AtLDOX_AT4g22880_selection-v2_guide-v5.fasta \
> data/v2g5_JOX-ANS-FLS-subset.fasta

: 1